In [7]:
import util
import model
import tensorflow as tf
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, make_scorer
from tensorflow.keras import datasets, layers, models, callbacks, regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# Specify number of epochs for the two models
epochs_drop = 10
epochs_l2 = 10

# Specify K for K-fold cross validaton
num_folds = 5

# Specify confidence level ( < 1)
cf_lvl = 0.95

In [9]:
train_images, train_labels, test_images, test_labels, image_shape = util.get_images()

In [16]:
def create_model_drop(
    filter_nos=[32, 32, 32],
    filter_size=(3, 3),
    dense_layers=[64],
    input_shape=image_shape,
    patience=5,
    dropoutRate=0.1,
    output_length=100
):
    """
    Args:
        filter_nos (tuple): number of filters in each Conv2D layer
        filter_size (tuple): size of Conv2D filters. Filters are assumed to have uniform sizes across layers.
        input_shape (tuple): shape of the input of the first layer
        dense_layer (tuple): number of hidden units in each Dense layer except the output layer. Output layer is hardcoded to the number of classes.
        patience (integer): number of epochs of no improvement until early stop triggers.
        dropoutRate (double < 1): rate of value drop
        output_length (integer): number of classes for output
    """
    # Would include a dropout parameter, but wasn't absolutely sure where those would be, though I would assume closer to the start.

    model = models.Sequential()
    callback = callbacks.EarlyStopping(monitor='loss',patience=patience) # EarlyStopping 

    for i, filter_no in enumerate(filter_nos):
        if i == 0:
            # first Conv2D requires input_shape
            model.add(
                layers.Conv2D(
                    filter_no, filter_size, activation="relu", input_shape=input_shape, kernel_regularizer='l2'
                )
            )
        else:
            model.add(layers.Conv2D(filter_no, filter_size, activation="relu"))

        if i + 1 < len(filter_nos):
            model.add(
                layers.MaxPooling2D((2, 2))
            )  # add MaxPooling layer to all but the last convolutional stack
            model.add(
                layers.Dropout(rate=dropoutRate)
            )  

    model.add(layers.Flatten())

    for dense_layer in dense_layers:
        model.add(layers.Dense(dense_layer, activation="relu"))

    model.add(layers.Dense(output_length))  # output layer

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [17]:
def create_model_l2(
    filter_nos=[32, 32, 32],
    filter_size=(3, 3),
    dense_layers=[64],
    input_shape=image_shape,
    patience=5,
    l2param=0.1,
    output_length=100
):
    """
    Args:
        filter_nos (tuple): number of filters in each Conv2D layer
        filter_size (tuple): size of Conv2D filters. Filters are assumed to have uniform sizes across layers.
        input_shape (tuple): shape of the input of the first layer
        dense_layer (tuple): number of hidden units in each Dense layer except the output layer. Output layer is hardcoded to the number of classes.
        patience (integer): number of epochs of no improvement until early stop triggers.
        l2param (double < 1): param given to L2
        output_length (integer): number of classes given to output
    """
    # Would include a dropout parameter, but wasn't absolutely sure where those would be, though I would assume closer to the start.

    model = models.Sequential()
    regularizers.L2(l2=l2param) #L2
    callback = callbacks.EarlyStopping(monitor='loss',patience=patience) # EarlyStopping 

    for i, filter_no in enumerate(filter_nos):
        if i == 0:
            # first Conv2D requires input_shape
            model.add(
                layers.Conv2D(
                    filter_no, filter_size, activation="relu", input_shape=input_shape, kernel_regularizer='l2'
                )
            )
        else:
            model.add(layers.Conv2D(filter_no, filter_size, activation="relu", kernel_regularizer='l2'))

        if i + 1 < len(filter_nos):
            model.add(
                layers.MaxPooling2D((2, 2))
            )  # add MaxPooling layer to all but the last convolutional stack

    model.add(layers.Flatten())

    for dense_layer in dense_layers:
        model.add(layers.Dense(dense_layer, activation="relu", kernel_regularizer='l2'))

    model.add(layers.Dense(output_length))  # output layer

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [18]:
stopper = callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
print("Dropoff\n")
classifier_drop = KerasClassifier(
    create_model_drop,
    filter_nos=(32, 64, 64),
    filter_size=(3, 3),
    dense_layers=(64,),
    input_shape=image_shape,
    patience=5,
    dropoutRate=0.1,
    output_length=100,
    epochs=epochs_drop,
    batch_size=32,
)

dropScores = cross_validate(
    estimator=classifier_drop, X=train_images, y=train_labels, cv=num_folds, scoring=make_scorer(accuracy_score)
)
print(dropScores["test_score"])

Dropoff



2024-07-10 22:57:09.863136: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2024-07-10 22:57:09.863216: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-10 22:57:09.863233: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c3618.swan.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2024-07-10 22:57:09.863485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 22:57:09.900260: I tensorflow/core/platform/profile_

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1250/1250 [==============================] - 27s 21ms/step - loss: 4.1752 - accuracy: 0.0600
Epoch 2/10
1250/1250 [==============================] - 26s 21ms/step - loss: 3.6233 - accuracy: 0.1461
Epoch 3/10
1250/1250 [==============================] - 26s 21ms/step - loss: 3.3237 - accuracy: 0.1990
Epoch 4/10
1250/1250 [==============================] 

In [ ]:
print("L2\n")
classifier_l2 = KerasClassifier(
    create_model_l2,
    filter_nos=(32, 64, 64),
    filter_size=(3, 3),
    dense_layers=(64,),
    input_shape=image_shape,
    patience=5,
    l2param=0.1,
    output_length=100,
    epochs=epochs_l2,
    batch_size=32,
)

l2Scores = dropScores = cross_validate(
    estimator=classifier_l2, X=train_images, y=train_labels, cv=num_folds, scoring=make_scorer(accuracy_score)
)
print(l2Scores["test_score"])

In [ ]:
diff = util.diff_scores(dropScores['test_score'], l2Scores['test_score'])
mean_diff = np.mean(diff)
std_diff = np.std(diff, ddof=(num_folds-1))
#cf_interval_heart = t.interval(cf_lvl, df, mean_diff_heart, std_diff_heart/np.sqrt(num_folds))
cf_interval_heart = t.interval(cf_lvl, (num_folds-1), loc=mean_diff, scale=std_diff/np.sqrt(num_folds))